# module init log parsing

To parse module init log from time stamp based log



In [1]:
import re
import json
import requests
import os
import errno

import logging

# A logger for this file
logger = logging.getLogger(__name__)

In [2]:
def _extract_module_init_data(file_name_url: str, logger=logger, working_dir: str="./", module_init_file: str="modules.json", notif_init_file: str="notifications.json"):

    """load module init data
    either from file or raw log from url
    extract data and save json file to the working_dir

    Parameters
    ----------
    file_name_url : str
        file name or url, if it url, then it is expected to starts with 'http'
    logger : logger
        logger
    working_dir : str, optional
        working directory, by default "./"
    module_init_file: str, optional
        module init file name, by default "modules.json", full path for saved file will be working_dir/module_init_file
    notif_init_file: str, optional
        notification init file name, by default "notifications.json", full path for saved file will be working_dir/notif_init_file

    Returns
    -------
    module_init_file_name: str
        full path of the saved module init file
    notif_init_file_name: str
        full path of the saved notification init file

    Raises
    ------
    FileNotFoundError
        if file_name_url is not a file or url
    requests.exceptions*
        if file_name_url is url and it is not accessible
    """

    read_url = True if file_name_url.startswith("http") else False
    if read_url:
        try:
            response = requests.get(file_name_url)
            lines = response.text
        except requests.exceptions.HTTPError as ex:
            logger.error("Http error: {}".format(ex))
            raise ex
        except requests.exceptions.ConnectionError as ex:
            logger.error("Error Connecting: {}".format(ex))
            raise ex
        except requests.exceptions.Timeout as ex:
            logger.error("Timeout Error: {}".format(ex))
            raise ex
        except requests.exceptions.RequestException as ex:
            logger.error("Exception {}".format(ex))
            raise ex
    else:
        try:
            with open(file_name_url) as f:
                lines = f.read()
        except FileNotFoundError as ex:
            logger.error("File not found: {}".format(file_name_url))
            raise ex

    # remove time stamp from each line
    # FROM: [1664319485.206378 0.0.0] pci_early: performing i2c/MUD initialization
    lines = re.sub(r'\[.*\] ', '', lines)

    MODULE_MARKERS = r'Time-chart data for module init = (.*?)}'
    NOTIF_MARKERS = r'Time-chart data for notifications = (.*?)}'

    module_init = re.search(MODULE_MARKERS, lines, re.DOTALL).group(1)
    module_init += "}" # add back the end marker

    notif_init = re.search(NOTIF_MARKERS, lines, re.DOTALL).group(1)
    notif_init += "}" # add back the end marker

    module_init = json.loads(module_init)
    notif_init = json.loads(notif_init)

    # check if the working_dir exists
    if not os.path.exists(working_dir):
        os.makedirs(working_dir)
    
    modules_init_file_name = os.path.join(working_dir, module_init_file)
    notificaiotns_init_file_name = os.path.join(working_dir, notif_init_file)

    # save files
    with open(modules_init_file_name, 'w') as f:
        json.dump(module_init, f, indent=4)


    with open(notificaiotns_init_file_name, 'w') as f:
        json.dump(notif_init, f, indent=4)
    
    logger.debug("Module init file saved to {}".format(modules_init_file_name))
    logger.debug("Notifications init file saved to {}".format(notificaiotns_init_file_name))

    return modules_init_file_name, notificaiotns_init_file_name 


In [3]:
# Main call
INPUT_FILE_URL = "http://palladium-jobs.fungible.local:8080/job/4297914/raw_file/odp/uartout0.0.txt"

module_init, notif_init = _extract_module_init_data(INPUT_FILE_URL, logger=logger)

In [4]:
stop

NameError: name 'stop' is not defined

In [ ]:
# input file
# INPUT_FILE = "posix-log.txt"
INPUT_FILE = "uartout0.0.txt"
INPUT_FILE_URL = "http://palladium-jobs.fungible.local:8080/job/4297914/raw_file/odp/uartout0.0.txt"
MODULE_INIT_FILE = "modules_from_log.json"
NOTIF_INIT_FILE = "notifications_from_log.json"

In [ ]:
READ_URL = True
READ_FILE = False

In [ ]:
response = requests.get(INPUT_FILE_URL)
lines = response.text

In [ ]:
if READ_FILE:
    # load text file
    with open(INPUT_FILE) as f:
        # lines = f.readlines()
        lines = f.read()
elif READ_URL:
    response = requests.get(INPUT_FILE_URL)
    lines = response.text
else:
    print("No input file or url specified")
    assert False


In [ ]:
# remove time stamp from each line
# FROM: [1664319485.206378 0.0.0] pci_early: performing i2c/MUD initialization
# TO  : pci_early: performing i2c/MUD initialization
# import re
# lines = [re.sub(r'\[.*\] ', '', line) for line in lines]

In [ ]:
# remove time stamp from each line
# FROM: [1664319485.206378 0.0.0] pci_early: performing i2c/MUD initialization
lines = re.sub(r'\[.*\] ', '', lines)

In [ ]:
MODULE_MARKERS = r'Time-chart data for module init = (.*?)}'

NOTIF_MARKERS = r'Time-chart data for notifications = (.*?)}'

In [ ]:
# with open('submission.tex') as f: print((re.search(r'% <SCPD_SUBMISSION_TAG>_2a(.*?)% <SCPD_SUBMISSION_TAG>_2a', f.read(), re.DOTALL)).group(1))

In [ ]:
module_init = re.search(MODULE_MARKERS, lines, re.DOTALL).group(1)
module_init += "}" # add back the end marker

In [ ]:
notif_init = re.search(NOTIF_MARKERS, lines, re.DOTALL).group(1)
notif_init += "}" # add back the end marker

In [ ]:
module_init = json.loads(module_init)

In [ ]:
notif_init = json.loads(notif_init)

In [ ]:
# save file
with open(MODULE_INIT_FILE, 'w') as f:
    json.dump(module_init, f, indent=4)


with open(NOTIF_INIT_FILE, 'w') as f:
    json.dump(notif_init, f, indent=4)
